In [179]:
import pandas as pd
import sklearn as sk
import seaborn as sns
import json
import copy
import re
import numpy as np
import scipy.optimize as opt

sns.set(style="darkgrid")

# Load an example dataset with long-form data
#fmri = sns.load_dataset("fmri")

# Plot the responses for different events and regions
#plt = sns.lineplot(x="timepoint", y="signal",
#             hue="region", style="event",
#             data=fmri)


#plt.show()

## Load geolocalized points for communes

In [180]:
geo_communes_columns = ["place","zipcode","zusatzziffer","commune","bfsnr","canton","X","Y","language"]
geo_communes = pd.read_csv("PLZO_CSV_WGS84.csv",sep=";",encoding="utf-8", names=geo_communes_columns, header=0)
# was encoding windows1252 (pandas encoding="cp1252"), now encoding unknown -> ?!?
print(geo_communes.shape)

# Only keep gemeinde:
#geo_communes = geo_communes[geo_communes.place==geo_communes.commune]
#print(geo_communes.shape)

# Drop thurgau for now
geo_communes = geo_communes[geo_communes.canton!="TG"]
print(geo_communes.shape)

geo_communes.head()

(4138, 9)
(3937, 9)


,place,zipcode,zusatzziffer,commune,bfsnr,canton,X,Y,language
0,Aeugst am Albis,8914,0,Aeugst am Albis,1,ZH,8.488,47.267,de
1,Aeugstertal,8914,2,Aeugst am Albis,1,ZH,8.494,47.283,de
2,Zwillikon,8909,0,Affoltern am Albis,2,ZH,8.431,47.288,de
3,Affoltern am Albis,8910,0,Affoltern am Albis,2,ZH,8.449,47.279,de
4,Bonstetten,8906,0,Bonstetten,3,ZH,8.468,47.316,de


There are many big cities with tons of zipcodes:

In [181]:
#geo_communes[geo_communes.place=="Zürich"]
geo_communes[geo_communes.place.duplicated()].place.unique()

array(['Wetzikon ZH', 'Winterthur', 'Zürich', 'Bern', 'Biel/Bienne',
       'Lyssach', 'Interlaken', 'Steffisburg', 'Thun', 'Hämikon',
       'Luzern', 'Wollerau', 'Cousset', 'Progens', 'Basel', 'Gächlingen',
       'Schaffhausen', 'Wilchingen', 'Oberegg', 'St. Gallen', 'Reute AR',
       'Montlingen', 'Wangs', 'Wil SG', 'Tiefencastel', 'Riom', 'Schnaus',
       'Ladir', 'Flerden', 'Innerferrera', 'Ramosch', 'Bever', 'Madulain',
       'Zuoz', 'Verdabbio', 'S. Vittore', 'Schiers', 'Stels', 'Aarau',
       'Baden', 'Sins', 'Murgenthal', 'Bellinzona', 'Aquila',
       'Chiggiogna', 'Locarno', 'Bedigliora', 'Piandera Paese',
       'Mezzovico', 'Ponte Capriasca', 'Corticiasca', 'Taverne',
       'Capolago', 'Medeglia', 'Lausanne', 'Mühlebach (Goms)',
       'Steinhaus', 'Fiesch', 'Saillon', 'Collombey', 'Monthey',
       'Baltschieder', 'St-Aubin-Sauges', 'La Cibourg', 'Le Locle',
       'Céligny', 'Genève', 'St-Ursanne'], dtype=object)

Group cities with multiple zip codes together and average their geo coordinates in a single point:

In [182]:
def averageMultipleZips(df):
    df.Y = df.Y.mean()
    df.X = df.X.mean()
    zipcodes = df.zipcode.values.tolist()
    zipcodes = [zipcodes for z in zipcodes]
    #print("zipcodes")
    #print(zipcodes)
    #print("df.zipcode")
    #print(df.zipcode)
    #print("df")
    #print(df)
    df.zipcode = zipcodes
    return df.iloc[0]

geo_communes = geo_communes.groupby('place', group_keys=False).apply(averageMultipleZips)

del geo_communes.index.name
geo_communes.head()

,place,zipcode,zusatzziffer,commune,bfsnr,canton,X,Y,language
Aarau,Aarau,"[5000, 5004]",0,Aarau,4001,AG,8.055,47.395,de
Aarau Rohr,Aarau Rohr,[5032],0,Aarau,4001,AG,8.083,47.405,de
Aarberg,Aarberg,[3270],0,Aarberg,301,BE,7.275,47.044,de
Aarburg,Aarburg,[4663],0,Aarburg,4271,AG,7.899,47.321,de
Aarwangen,Aarwangen,[4912],0,Aarwangen,321,BE,7.770,47.241,de


## Load cleaned communes population data

In [183]:
columns_communes = ["name","canton","url","firstmention","hab_year","raw_hab_year","notes"]

with open('../2_pop_extrapolation/communes_units_converted.json', 'r') as cf:
    communes = json.load(cf)
    
dfcommunes = pd.DataFrame(communes)[columns_communes]
print(dfcommunes.shape)


# Drop thurgau for now
dfcommunes = dfcommunes[dfcommunes.canton!="TG"]
print(dfcommunes.shape)

(2242, 7)
(2162, 7)


### Hand correction for communes:

In [184]:
# correct canton value for Basel-Stadt and Moutier
dfcommunes.loc[dfcommunes.name=="Moutier","canton"]="BE"
dfcommunes.loc[dfcommunes.name=="Basel-Stadt","canton"]="BS"
# Rename communes to their nom d'usage in french:
dfcommunes.loc[dfcommunes.name=="Sitten","name"]="Sion"
dfcommunes.loc[dfcommunes.name=="Neuenburg","name"]="Neuchâtel"

In [185]:
pd.reset_option('display.max_rows')
dfcommunes.to_csv("communes.csv", sep=";")

## Reshape DataFrame to 1 line per datapoint

In [186]:
columns_communes_datapoints = ["year","pop","unit","name","canton","url","firstmention","hab_year","notes"]
communes_datapoints = []

for commune in communes:
    for hy in commune["hab_year"]:
        hy_dict = copy.deepcopy(commune)
        hy_dict["year"] =  hy["year"]
        hy_dict["pop"] =  hy["pop"]
        hy_dict["unit"] = hy["unit"] if "unit" in hy else "undefined"
        communes_datapoints.append(hy_dict)

dfcommunes_datapoints = pd.DataFrame(communes_datapoints)[columns_communes_datapoints]
dfcommunes_datapoints = dfcommunes_datapoints.drop(columns=["hab_year"])
print(dfcommunes_datapoints.shape)

# Drop thurgau for now
dfcommunes_datapoints = dfcommunes_datapoints[dfcommunes_datapoints.canton!="TG"]
print(dfcommunes_datapoints.shape)

dfcommunes_datapoints

(13423, 8)
(13141, 8)


,year,pop,unit,name,canton,url,firstmention,notes
74,1416,600,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
75,1764,1751,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
76,1803,2001,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
77,1850,2054,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
78,1900,3025,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
79,1930,3840,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
80,1950,3381,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
81,1980,2872,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
82,1990,3110,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."
83,2000,2949,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit..."


In [187]:
dfcommunes_datapoints.to_csv("communes_datapoints.csv", sep=";")

## TODO: merge communes and geo_communes using Hungarian algorithm
https://docs.scipy.org/doc/scipy-0.19.0/reference/generated/scipy.optimize.linear_sum_assignment.html

##### Create the fuzzy merke_keys:

In [188]:
from jellyfish import jaro_distance

#create unique list of names
cantons = [ c for c in dfcommunes.canton.unique() if c!="MA"]
test_canton = "ZH"

communes_per_canton = {}
geo_communes_per_canton = {}
distance_matrix_per_canton = {}
merge_keys_per_canton = {}
for canton in cantons:
    communes_per_canton[canton] = dfcommunes.name[dfcommunes.canton == canton].unique()
    geo_communes_per_canton[canton] = geo_communes.place[geo_communes.canton == canton].unique()
    
    # do the fuzzy merge_keys
    #if canton==test_canton:
    def distance(i, j):
        return 1-jaro_distance(communes_per_canton[canton][np.int(i)], geo_communes_per_canton[canton][np.int(j)])

    jaroDistanceProxy = np.vectorize(distance)
    distance_matrix_per_canton[canton] = np.fromfunction(
        jaroDistanceProxy,
        shape=(len(communes_per_canton[canton]),
               len(geo_communes_per_canton[canton])))

    fuzzy_merge = opt.linear_sum_assignment(distance_matrix_per_canton[canton])
    merge_keys_per_canton[canton] = {
        communes_per_canton[canton][fuzzy_merge[0][i]]:
        geo_communes_per_canton[canton][fuzzy_merge[1][i]]
        for i in range(0,len(fuzzy_merge[0]))
    }
        
    
    
    

print(communes_per_canton[test_canton])
print(geo_communes_per_canton[test_canton])
merge_keys_per_canton[test_canton]

['Adliswil' 'Aeugst am Albis' 'Affoltern am Albis' 'Altikon' 'Andelfingen'
 'Bachenbülach' 'Bachs' 'Bäretswil' 'Bassersdorf' 'Bauma' 'Benken (ZH)'
 'Berg am Irchel' 'Birmensdorf (ZH,' 'Bonstetten' 'Boppelsen' 'Brütten'
 'Bubikon' 'Buch am Irchel' 'Buchs (ZH)' 'Bülach' 'Dachsen' 'Dägerlen'
 'Dällikon' 'Dänikon' 'Dättlikon' 'Dielsdorf' 'Dietikon' 'Dietlikon'
 'Dinhard' 'Dorf' 'Dübendorf' 'Dürnten' 'Egg (ZH)' 'Eglisau' 'Elgg'
 'Ellikon an der Thur' 'Elsau' 'Embrach' 'Erlenbach (ZH)' 'Fällanden'
 'Fehraltorf' 'Feuerthalen' 'Fischenthal' 'Flaach' 'Flurlingen'
 'Freienstein-Teufen' 'Geroldswil' 'Glattfelden' 'Gossau (ZH)'
 'Greifensee' 'Aesch bei Birmensdorf' 'Adlikon' 'Grüningen' 'Hagenbuch'
 'Hausen am Albis' 'Hedingen' 'Henggart' 'Herrliberg' 'Hettlingen'
 'Hinwil' 'Hirzel' 'Hittnau' 'Hochfelden' 'Hofstetten (ZH)'
 'Hombrechtikon' 'Horgen' 'Höri' 'Humlikon' 'Hüntwangen' 'Hütten'
 'Hüttikon' 'Illnau-Effretikon' 'Kappel am Albis' 'Kilchberg (ZH)'
 'Kleinandelfingen' 'Kloten' 'Knonau' 'Küsna

{'Adliswil': 'Adliswil',
 'Aeugst am Albis': 'Aeugst am Albis',
 'Affoltern am Albis': 'Affoltern am Albis',
 'Altikon': 'Altikon',
 'Andelfingen': 'Andelfingen',
 'Bachenbülach': 'Bachenbülach',
 'Bachs': 'Bachs',
 'Bäretswil': 'Bäretswil',
 'Bassersdorf': 'Bassersdorf',
 'Bauma': 'Bauma',
 'Benken (ZH)': 'Benken ZH',
 'Berg am Irchel': 'Berg am Irchel',
 'Birmensdorf (ZH,': 'Birmensdorf ZH',
 'Bonstetten': 'Bonstetten',
 'Boppelsen': 'Boppelsen',
 'Brütten': 'Brütten',
 'Bubikon': 'Bubikon',
 'Buch am Irchel': 'Buch am Irchel',
 'Buchs (ZH)': 'Buchs ZH',
 'Bülach': 'Bülach',
 'Dachsen': 'Dachsen',
 'Dägerlen': 'Dägerlen',
 'Dällikon': 'Dällikon',
 'Dänikon': 'Nänikon',
 'Dättlikon': 'Dättlikon',
 'Dielsdorf': 'Dielsdorf',
 'Dietikon': 'Dietikon',
 'Dietlikon': 'Dietlikon',
 'Dinhard': 'Dinhard',
 'Dorf': 'Dorf',
 'Dübendorf': 'Dübendorf',
 'Dürnten': 'Dürnten',
 'Egg (ZH)': 'Egg b. Zürich',
 'Eglisau': 'Eglisau',
 'Elgg': 'Elgg',
 'Ellikon an der Thur': 'Ellikon an der Thur',
 'Elsau

#### Do the fuzzy merge:
...on dfcommunes:

In [189]:
dfcommunes["place"] = ""

for index, row in dfcommunes.iterrows():
    if row["canton"]!="MA" and row["name"] in merge_keys_per_canton[row["canton"]]:
        dfcommunes.loc[index,"place"] = merge_keys_per_canton[row["canton"]][row["name"]] 
    #print(dfcommunes.iloc[index])

#dfcommunes.canton=="MA"
result_communes = pd.merge(dfcommunes, geo_communes, on='place', how='left')
result_communes.to_csv("communes_geo.csv", sep=";")

result_communes

,name,canton_x,url,firstmention,hab_year,raw_hab_year,notes,place,zipcode,zusatzziffer,commune,bfsnr,canton_y,X,Y,language
0,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"[{'year': 1416, 'pop': 600, 'unit': 'Einw', 'o...","[{'year': 1416, 'pop': 120, 'unit': 'Haushalte...","hab+foyer data!, unclear population count (wit...",Château-d'Oex,[1660],0,Château-d'Oex,5841,VD,7.1290,46.4730,fr
1,Lutry,VD,/Articles/002420/?language=de,908,"[{'year': 1850, 'pop': 2011, 'unit': 'Einw'}, ...","[{'year': 1850, 'pop': 2011, 'unit': 'Einw'}, ...","hab+foyer data!, with or without Savigny,",Lutry,[1095],0,Lutry,5606,VD,6.6860,46.5030,fr
2,Ballaigues,VD,/Articles/002528/?language=de,1228,"[{'year': 1416, 'pop': 50, 'unit': 'Einw', 'or...","[{'year': 1416, 'pop': 10, 'unit': 'Haushalte'...","hab+foyer data!,",Ballaigues,[1338],0,Ballaigues,5744,VD,6.4150,46.7290,fr
3,Belmont-sur-Yverdon,VD,/Articles/002623/?language=de,1154,"[{'year': 1409, 'pop': 155, 'unit': 'Einw', 'o...","[{'year': 1409, 'pop': 31, 'unit': 'Haushalte'...","hab+foyer data!,",Belmont-sur-Yverdon,[1432],0,Belmont-sur-Yverdon,5902,VD,6.6240,46.7460,fr
4,Borex,VD,/Articles/002498/?language=de,1135,"[{'year': 1429, 'pop': 70, 'unit': 'Einw', 'or...","[{'year': 1429, 'pop': 14, 'unit': 'Feuerstätt...","hab+foyer data!,",Borex,[1277],0,Borex,5706,VD,6.1780,46.3790,fr
5,Ependes (VD),VD,/Articles/002635/?language=de,1154,"[{'year': 1404, 'pop': 85, 'unit': 'Einw', 'or...","[{'year': 1404, 'pop': 17, 'unit': 'Haushalte'...","hab+foyer data!,",Ependes VD,[1434],0,Ependes (VD),5914,VD,6.6080,46.7450,fr
6,Lignerolle,VD,/Articles/002538/?language=de,1160,"[{'year': 1416, 'pop': 300, 'unit': 'Einw', 'o...","[{'year': 1416, 'pop': 60, 'unit': 'Feuerstätt...","hab+foyer data!,",Lignerolle,[1357],0,Lignerolle,5755,VD,6.4560,46.7400,fr
7,Premier,VD,/Articles/002542/?language=de,1316,"[{'year': 1396, 'pop': 75, 'unit': 'Einw', 'or...","[{'year': 1396, 'pop': 15, 'unit': 'Feuerstätt...","hab+foyer data!,",Premier,[1324],0,Premier,5759,VD,6.4450,46.7040,fr
8,Pully,VD,/Articles/002412/?language=de,994,"[{'year': 1764, 'pop': 626, 'unit': 'Einw'}, {...","[{'year': 1764, 'pop': 626, 'unit': 'Einw'}, {...","hab+foyer data!,",Pully,[1009],0,Pully,5590,VD,6.6620,46.5090,fr
9,Suchy,VD,/Articles/002650/?language=de,885,"[{'year': 1409, 'pop': 120, 'unit': 'Einw', 'o...","[{'year': 1409, 'pop': 24, 'unit': 'Feuerstätt...","hab+foyer data!,",Suchy,[1433],0,Suchy,5929,VD,6.5990,46.7220,fr


...on dfcommunes_datapoints:

In [190]:
dfcommunes_datapoints["place"] = ""

for index, row in dfcommunes_datapoints.iterrows():
    if row["canton"]!="MA" and row["name"] in merge_keys_per_canton[row["canton"]]:
        dfcommunes_datapoints.loc[index,"place"] = merge_keys_per_canton[row["canton"]][row["name"]] 
    #print(dfcommunes.iloc[index])

#dfcommunes.canton=="MA"
result_communes_datapoints = pd.merge(dfcommunes_datapoints, geo_communes, on='place', how='left')
result_communes_datapoints.to_csv("communes_datapoints_geo.csv", sep=";")

result_communes_datapoints

,year,pop,unit,name,canton_x,url,firstmention,notes,place,zipcode,zusatzziffer,commune,bfsnr,canton_y,X,Y,language
0,1416,600,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit...",Château-d'Oex,[1660],0.0,Château-d'Oex,5841.0,VD,7.1290,46.4730,fr
1,1764,1751,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit...",Château-d'Oex,[1660],0.0,Château-d'Oex,5841.0,VD,7.1290,46.4730,fr
2,1803,2001,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit...",Château-d'Oex,[1660],0.0,Château-d'Oex,5841.0,VD,7.1290,46.4730,fr
3,1850,2054,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit...",Château-d'Oex,[1660],0.0,Château-d'Oex,5841.0,VD,7.1290,46.4730,fr
4,1900,3025,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit...",Château-d'Oex,[1660],0.0,Château-d'Oex,5841.0,VD,7.1290,46.4730,fr
5,1930,3840,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit...",Château-d'Oex,[1660],0.0,Château-d'Oex,5841.0,VD,7.1290,46.4730,fr
6,1950,3381,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit...",Château-d'Oex,[1660],0.0,Château-d'Oex,5841.0,VD,7.1290,46.4730,fr
7,1980,2872,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit...",Château-d'Oex,[1660],0.0,Château-d'Oex,5841.0,VD,7.1290,46.4730,fr
8,1990,3110,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit...",Château-d'Oex,[1660],0.0,Château-d'Oex,5841.0,VD,7.1290,46.4730,fr
9,2000,2949,Einw,Château-d'Œx,VD,/Articles/002593/?language=de,1080,"hab+foyer data!, unclear population count (wit...",Château-d'Oex,[1660],0.0,Château-d'Oex,5841.0,VD,7.1290,46.4730,fr


In [191]:
print(result_communes.shape)
print(dfcommunes.shape)

(2162, 16)
(2162, 8)


In [192]:
print("Nb of communes that didn't get a geo localisation:")
print(np.sum(dfcommunes.place==""))
dfcommunes[dfcommunes.place==""]

Nb of communes that didn't get a geo localisation:
0


,name,canton,url,firstmention,hab_year,raw_hab_year,notes,place


'Sion'